In [1]:
import requests
import json
import pandas as pd

In [2]:
AUTH_URL = "https://akvofoundation.eu.auth0.com/oauth/token"
SURVEY_URL = "https://api-auth0.akvo.org/flow/orgs/maep/surveys/806432680"
MONITORING_FORM_NAME = "Example Monitoring"

In [3]:
USERNAME = ""
PASSWORD = ""

In [4]:
r = requests.Session()

In [5]:
def get_token():                                                                                                                                                     
    data = {                                                                                                                                                                                                         
        "client_id": "S6Pm0WF4LHONRPRKjepPXZoX1muXm1JS",                                                                                                                                                             
        "username": USERNAME,                                                                                                                                                                                        
        "password": PASSWORD,                                                                                                                                                                     
        "grant_type": "password",                                                                                                                                                                                    
        "scope": "offline_access"                                                                                                                                                                                    
    }                                                                                                                                                                                                                
    req = r.post(AUTH_URL, data=data)                                                                                                                                                                             
    if req.status_code != 200:                                                                                                                                                                                       
        print("LOGIN ERROR")                                                                                                                                                              
    return req.json()

In [6]:
def get_data(target_url, headers):
    res = r.get(target_url, headers=headers)
    if res.status_code != 200:                                                                                                                                                                                       
        print("FETCH DATA ERROR")
    return res.json()

In [7]:
def get_headers(refresh_token):                                                                             
    login = {                                                                                            
        'client_id': 'S6Pm0WF4LHONRPRKjepPXZoX1muXm1JS',                                                 
        'grant_type': 'refresh_token',                                                                   
        'refresh_token': refresh_token,                                                                          
        'scope': 'openid email'                                                                          
    }                                                                                                    
    req = r.post(AUTH_URL, data=login)                          
    if req.status_code != 200:                                                                           
        return False                                                                                     
    return {                                                                                             
        'Content-Type': 'application/json',                                                              
        'Accept': 'application/vnd.akvo.flow.v2+json',                                                   
        'Authorization': 'Bearer {}'.format(req.json().get('id_token'))                                  
    }

In [8]:
def fetch_all(url, headers, formInstances=[]):                                                                                                                                                                       
    data = get_data(url, headers)                                                                                                                                                                                    
    next_url = data.get('nextPageUrl')                                                                                                                                                                               
    data = data.get('formInstances')                                                                                                                                                                                 
    if data:                                                                                                                                                                                                         
        for d in data:                                                                                                                                                                                               
            formInstances.append(d)                                                                                                                                                                                  
        if next_url:                                                                                                                                                                                                 
            fetch_all(next_url, headers, formInstances)                                                                                                                                                              
    return formInstances 

In [9]:
def data_handler(data, qType):                                                                                                                                                                                       
    if data:                                                                                                                                                                                                         
        if qType in [                                                                                                                                                                                                
                'FREE_TEXT', 'NUMBER', 'BARCODE', 'DATE', 'GEOSHAPE', 'SCAN',                                                                                                                                        
                'CADDISFLY'                                                                                                                                                                                          
        ]:                                                                                                                                                                                                           
            return data                                                                                                                                                                                              
        if qType == 'OPTION':                                                                                                                                                                                        
            return handle_list(data, "text")                                                                                                                                                                         
        if qType == 'CASCADE':                                                                                                                                                                                       
            return handle_list(data, "name")                                                                                                                                                                         
        if qType in ['PHOTO', 'VIDEO']:                                                                                                                                                                              
            return data.get('filename')                                                                                                                                                                              
        if qType == 'VIDEO':                                                                                                                                                                                         
            return data.get('filename')                                                                                                                                                                              
        if qType == 'GEO':                                                                                                                                                                                           
            return {'lat': data.get('lat'), 'long': data.get('long')}                                                                                                                                                
        if qType == 'SIGNATURE':                                                                                                                                                                                     
            return data.get("name")                                                                                                                                                                                  
    return None

In [10]:
def handle_list(data, target):                                                                                                                                                                                       
    response = []                                                                                                                                                                                                    
    for value in data:                                                                                                                                                                                               
        response.append(value.get(target))                                                                                                                                                                       
    return "|".join(response)

In [11]:
def get_page(form_instance_url, form_definition, refresh_token):                                                                                                                                               
    headers = get_headers(refresh_token)                                                                                                                                                                        
    collections = fetch_all(form_instance_url, headers)                                                                                                                                                          
    results = []                                                                                                                                                                                                     
    for col in collections:                                                                                                                                                                                          
        dt = {}                                                                                                                                                                                                      
        for c in col:                                                                                                                                                                                                
            if c != 'responses':                                                                                                                                                                                     
                dt.update({c: col[c]})                                                                                                                                                                               
            else:                                                                                                                                                                                                    
                for g in form_definition:                                                                                                                                                                            
                    answers = col.get(c)                                                                                                                                                                             
                    answers = answers.get(g['id']) if answers else None
                    for q in g['questions']:                                                                                                                                                                         
                        d = None
                        try:
                            a = answers[0].get(q['id'])
                            d = data_handler(a, q['type'])                                                                                                                                                           
                        except Exception as e:                                                                                                                                                                       
                            pass                                                                                                                                                                         
                        # n = "{}|{}".format(q['id'], q['name'])                                                                                                                                                       
                        dt.update({q['name']: d})                                                                                                                                                                            
        results.append(dt)                                                                                                                                                                                           
    return results

In [12]:
refresh_token = get_token().get("refresh_token")
headers = get_headers(refresh_token)
surveys = get_data(SURVEY_URL, headers)

In [13]:
registration_form_id = surveys.get("registrationFormId")
monitoring_forms = list(filter(lambda x: x.get("id") != registration_form_id, surveys.get("forms")))

In [14]:
data = {}

In [15]:
for form in surveys.get("forms"):
    form_id = form.get("id")
    form_definition = form.get("questionGroups")
    collections = get_page(form.get("formInstancesUrl"), form_definition, refresh_token)
    data.update({form_id: collections})

In [16]:
## DATA COLLECTIONS ENDS HERE

In [17]:
def get_monitoring_id_by_name(name:str):
    target = list(filter(lambda x: x.get("name").lower() == name.lower(), monitoring_forms))
    if len(target):
        return target[0].get("id")
    else:
        print(f"NOT FOUND: {name}")

In [18]:
monitoring_form_id = get_monitoring_id_by_name(MONITORING_FORM_NAME)
registration_data = data.get(registration_form_id)
monitoring_data = data.get(monitoring_form_id)

In [19]:
registration = pd.DataFrame(registration_data)
monitoring = pd.DataFrame(monitoring_data)

In [20]:
def splitter(x, pos):
    return x.split("|")[pos]

In [22]:
registration["DIVISION"] = registration["Address"].apply(lambda x: splitter(x, 0))

In [24]:
merged = monitoring.merge(registration, on='dataPointId', suffixes=('_registration', '_monitoring'))

In [26]:
aggregation = registration.groupby(["DIVISION"], as_index = False)["Age"].agg(
    ['min', 'max', 'sum']).to_dict("index")

In [28]:
pd.DataFrame(aggregation)

,ABOMEY,ADJA-OUERE,ADJOHOUN,AGBANGNIZOUN,BANIKOARA,BANTE,COVE,DANGBO,PERERE
min,34.0,23.0,30.0,30.0,30.0,25.0,35.0,25.0,34.0
max,34.0,23.0,30.0,30.0,30.0,25.0,35.0,25.0,34.0
sum,34.0,23.0,30.0,30.0,30.0,25.0,35.0,25.0,34.0
